# We vary the capacity of the Bayesian model by changing `hidden_size` parameter, trying to predict the brownian trajectories. We hope to see that, as the size of the model grows, it tends to overfit and detects difference between two very similar datasets, both of a brownian particle moving "forward", when we'd expect to see no difference.

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import torch

In [ ]:
from brownian_datagen import BrownianDatagen, create_dataholder_forward_same_as_backward
from bayesian_varinf import train_fb_n_times, load_learning_curves

In [ ]:
def get_save_dir_prefix(hidden_size: int) -> str:
    save_dir_prefix = "20230724_preds/brownian_varinf_discriminate_identical_nonoise/"
    return os.path.join(save_dir_prefix, f"hidden_size={hidden_size:02}/")

In [ ]:
brownian = BrownianDatagen(kBT=0.03, γ=1., k=1., λ_τ=5., τ=10.)
dt = create_dataholder_forward_same_as_backward(brownian, numParticles=50,
                                                rng_seed1=42, rng_seed2=43)
brownian.visualize()

In [ ]:
for hidden_size in (16, 8, 4):
    save_dir_prefix = get_save_dir_prefix(hidden_size)
    if not os.path.isdir(save_dir_prefix):
        train_fb_n_times(dt,
            num_runs=100,
            hidden_size=hidden_size,
            num_epochs=4000,
            save_metrics_every_n_epochs=100,
            train_test_split_ratio=None,
            save_dir_prefix=save_dir_prefix,
            torch_device="cuda:0" if torch.cuda.is_available() else None)
    else:
        print(f"Directory '{save_dir_prefix}' exists, assuming the previous results are reusable")

## Same but with much more noise

In [ ]:
def get_save_dir_prefix(hidden_size: int) -> str:
    save_dir_prefix = "20230724_preds/brownian_varinf_discriminate_identical_noisy/"
    return os.path.join(save_dir_prefix, f"hidden_size={hidden_size:02}/")

In [ ]:
brownian = BrownianDatagen(kBT=1., γ=1., k=1., λ_τ=5., τ=10.)
dt = create_dataholder_forward_same_as_backward(brownian, numParticles=50,
                                                rng_seed1=42, rng_seed2=43)
brownian.visualize()

In [ ]:
for hidden_size in (16, 8, 4):
    save_dir_prefix = get_save_dir_prefix(hidden_size)
    if not os.path.isdir(save_dir_prefix):
        train_fb_n_times(dt,
            num_runs=100,
            hidden_size=hidden_size,
            num_epochs=4000,
            save_metrics_every_n_epochs=100,
            train_test_split_ratio=None,
            save_dir_prefix=save_dir_prefix,
            torch_device="cuda:0" if torch.cuda.is_available() else None)
    else:
        print(f"Directory '{save_dir_prefix}' exists, assuming the previous results are reusable")